In [29]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVR
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt


df = pd.read_csv('processed.csv', index_col=0)

X = df.drop(labels=['Price'], axis=1, inplace=False)
y = df['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Linear Regression

In [12]:
lin_reg = LinearRegression()



model = lin_reg.fit(X_train, y_train)

train_cross_val = cross_val_score(lin_reg, X_train, y_train, cv=10, scoring='r2')


In [9]:
model.score(X_test, y_test)

0.6520112018578806

In [13]:
train_cross_val

array([0.64486988, 0.56465272, 0.75150363, 0.65785089, 0.61113381,
       0.45854438, 0.67780822, 0.67028887, 0.59222983, 0.56717359])

### DT

In [25]:
DT = DecisionTreeRegressor(max_depth =11) ## since it seems the first 11 features capture good variance for PCA

model = DT.fit(X_train, y_train)

train_cross_val = cross_val_score(DT, X_train, y_train, cv=10, scoring='r2')


In [26]:
model.score(X_test, y_test)

0.6467050040400648

In [27]:
train_cross_val

array([0.54510671, 0.48132491, 0.55131899, 0.63051534, 0.60286138,
       0.33922575, 0.60750238, 0.60418064, 0.35650239, 0.5737718 ])

### Random Forest

In [30]:
RF = RandomForestRegressor(max_depth=11)

model = RF.fit(X_train, y_train)

train_cross_val = cross_val_score(RF, X_train, y_train, cv=10, scoring='r2')


In [31]:
model.score(X_test, y_test)

0.7384679345561728

In [32]:
train_cross_val

array([0.69384234, 0.64422395, 0.78292521, 0.67822768, 0.71079064,
       0.51555641, 0.73165946, 0.69938051, 0.61373579, 0.60718916])

### SVM